In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import keras
from keras import layers
import rdkit
from rdkit import Chem, rdBase
from rdkit.Chem import AllChem
from rdkit import RDLogger
from tqdm import tqdm
from tqdm.keras import TqdmCallback
from collections import Counter # bag of elements?
import pubchempy as pcp

In [4]:
polymer_data_raw = pd.read_csv('simulation-trajectory-aggregate.csv')
polymer_data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6270 entries, 0 to 6269
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Trajectory ID             6270 non-null   int64  
 1   Mol smiles                6270 non-null   object 
 2   Molality                  6270 non-null   float64
 3   Monomer Molecular Weight  6270 non-null   float64
 4   Degree of Polymerization  6270 non-null   float64
 5   Density                   6270 non-null   float64
 6   CONDUCTIVITY              6270 non-null   float64
 7   TFSI Diffusivity          6270 non-null   float64
 8   Li Diffusivity            6270 non-null   float64
 9   Poly Diffusivity          6270 non-null   float64
 10  Transference Number       6270 non-null   float64
dtypes: float64(9), int64(1), object(1)
memory usage: 539.0+ KB


In [5]:
polymer_data_raw.head()

,Trajectory ID,Mol smiles,Molality,Monomer Molecular Weight,Degree of Polymerization,Density,CONDUCTIVITY,TFSI Diffusivity,Li Diffusivity,Poly Diffusivity,Transference Number
0,9425,COCC(CNCC(CF)OC(=O)[Au])O[Cu],1.4005,467.71,19.0,1.3494,0.000076,3.229400e-08,1.260800e-08,1.526400e-08,-0.162760
1,9426,O=C(CCNC(=O)COC(=O)[Au])NCCN[Cu],1.4735,475.72,13.0,1.4561,0.000070,1.317300e-08,1.105800e-08,8.636900e-09,0.317530
2,9427,NC(=O)C(COC(=O)[Au])NC(=O)CCO[Cu],1.4422,462.68,17.0,1.5336,0.000104,1.397400e-08,1.952200e-08,9.183200e-09,0.529740
3,9428,CC(COC(=O)[Au])COC(=O)C(C)(C)CO[Cu],1.4327,476.75,16.0,1.2767,0.000027,2.421000e-08,1.192500e-08,1.402700e-08,-0.319160
4,9429,COC(=O)CC(=O)NC(CO[Cu])COC(=O)[Au],1.4680,477.69,26.0,1.4896,0.000038,1.349100e-08,6.582900e-09,8.672500e-09,0.080645


In [6]:
count_invalid = 0
for index,smile in polymer_data_raw['Mol smiles'].items():
    # print(f'Smiles {i}: {smile}')

    try:
        mol = Chem.MolFromSmiles(smile)
        if mol is None: # drop entry if invalid smiles string
            print(f"Warning: RDKit returned None for SMILES: {smile}")
            count_invalid += 1
    except ArgumentError as e:
        print(f"Error parsing SMILES '{smile}': {e}")

if count_invalid != 0:
    print(f'{count_invalid} invalid SMILES strings.')
else:
    print('All SMILES strings are valid!')

All SMILES strings are valid!


In [24]:
fp_radius = 3
nb = 1024

molobj = [Chem.MolFromSmiles(smile) for smile in polymer_data_raw['Mol smiles']]
fingerprints = [AllChem.GetMorganFingerprintAsBitVect(mol,fp_radius,nBits=nb) for mol in molobj]
polymer_data_raw['Morgan Fingerprint'] = fingerprints

In [25]:
polymer_data_raw.head()

,Trajectory ID,Mol smiles,Molality,Monomer Molecular Weight,Degree of Polymerization,Density,CONDUCTIVITY,TFSI Diffusivity,Li Diffusivity,Poly Diffusivity,Transference Number,Morgan Fingerprint
0,9425,COCC(CNCC(CF)OC(=O)[Au])O[Cu],1.4005,467.71,19.0,1.3494,0.000076,3.229400e-08,1.260800e-08,1.526400e-08,-0.162760,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,9426,O=C(CCNC(=O)COC(=O)[Au])NCCN[Cu],1.4735,475.72,13.0,1.4561,0.000070,1.317300e-08,1.105800e-08,8.636900e-09,0.317530,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,9427,NC(=O)C(COC(=O)[Au])NC(=O)CCO[Cu],1.4422,462.68,17.0,1.5336,0.000104,1.397400e-08,1.952200e-08,9.183200e-09,0.529740,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,9428,CC(COC(=O)[Au])COC(=O)C(C)(C)CO[Cu],1.4327,476.75,16.0,1.2767,0.000027,2.421000e-08,1.192500e-08,1.402700e-08,-0.319160,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,9429,COC(=O)CC(=O)NC(CO[Cu])COC(=O)[Au],1.4680,477.69,26.0,1.4896,0.000038,1.349100e-08,6.582900e-09,8.672500e-09,0.080645,"[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [7]:
element_list = []
i_skip = False
for index,smile in polymer_data_raw['Mol smiles'].items():
    # print(smile)
    for i,char in enumerate(smile):
        if i_skip:
            i_skip = False
            continue
        if char.isalpha():
            try:
                if smile[i+1].islower():
                    element_check = smile[i:i+2]
                    i_skip = True
                else:
                    element_check = char
            except:
                element_check = char
            if element_check not in element_list:
                # print(element_check)
                element_list.append(element_check)
print(element_list)

['C', 'O', 'N', 'F', 'Au', 'Cu', 'S', 'Cl', 'Si', 'P', 'H']


In [8]:
bracket_list = []
i_skip = False
element_check = None
for index,smile in polymer_data_raw['Mol smiles'].items():
    # print(smile)
    for i,char in enumerate(smile):
        if char == '[':
            for j,char2 in enumerate(smile[i:]):
                if char2 == ']':
                    element_check = smile[i:i+j+1]
                    # print(element_check)
                    break
        if element_check is not None and element_check not in bracket_list:
            # print(element_check)
            bracket_list.append(element_check)
print(bracket_list)

['[Au]', '[Cu]', '[Si]', '[PH]', '[N+]', '[O-]']


In [10]:
num_pubchem = 0

for smile in tqdm(polymer_data_raw['Mol smiles'][::100]):
    results = pcp.get_compounds(smile, "smiles")
    if str(results[0]) != "Compound()":
        num_pubchem+=1
        print(results)
print(num_pubchem)

100%|██████████| 63/63 [00:23<00:00,  2.63it/s]

0


In [ ]:
raw = np.array([])
clean = np.array([])
for index,smile in polymer_data_raw['Mol smiles'].items():
    # print(smile)
    raw = np.append(raw,[smile])
    clean = np.append(clean,[smile.replace("[Au]", "").replace("[Cu]", "")])

property_list = ['MolecularWeight','TPSA']

for old,new in tqdm(zip(raw[::2],clean[::2])):
    try:
        x = pcp.get_compounds(old,'smiles')
        y = pcp.get_compounds(new,'smiles')
    except:
        pass
    if str(y) != '[Compound()]':
        p = pcp.get_properties(property_list,y[0].cid,'cid')
        print(y, new, old)
        print(p)

16it [00:13,  1.01it/s]

[Compound(18959637)] CC(CNC(=O))N CC(CNC(=O)[Au])N[Cu]
[{'CID': 18959637, 'MolecularWeight': '102.14', 'TPSA': 55.1}]


17it [00:14,  1.05it/s]

[Compound(129022519)] CC(C)C(N)C(=O)NCCOC(=O) CC(C)C(N[Cu])C(=O)NCCOC(=O)[Au]
[{'CID': 129022519, 'MolecularWeight': '188.22', 'TPSA': 81.4}]


26it [00:21,  1.26it/s]

[Compound(80309110)] CCC(CCO)CNC(=O) CCC(CCO[Cu])CNC(=O)[Au]
[{'CID': 80309110, 'MolecularWeight': '145.20', 'TPSA': 49.3}]


32it [00:25,  1.25it/s]


In [21]:
compound = pcp.get_compounds('CC(CNC(=O))N','smiles')
print(compound[0].cid)

18959637


In [ ]:
fp_radius = 3
nb = 1024

sample_smile = polymer_data_raw['Mol smiles'][0]
mol = Chem.MolFromSmiles(sample_smile)
print(sample_smile)

fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol,fp_radius,nBits=nb)


COCC(CNCC(CF)OC(=O)[Au])O[Cu]
